In [1]:
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

import tensorflow as tf

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img
%matplotlib inline

import tensorflow_hub as thub
import tensorflow.keras.backend as K

from lib.data_utils import *

from art.classifiers import KerasClassifier
from art.attacks import FastGradientMethod
import art

In [2]:
import pandas
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [3]:
def sample_by_class(data, labels, num_samples=100):
    sample_data = []
    sample_labels = []
    unq_labels = list(range(labels.shape[1]))
    for label in unq_labels:
        idx = labels[:,label]==1
        sample_set = data[idx][0:num_samples].copy()
        label_set = labels[idx][0:num_samples].copy()
        sample_data.append(sample_set)
        sample_labels.append(label_set)
    
    sample_data = np.concatenate(sample_data)
    sample_labels = np.concatenate(sample_labels)
    print(sample_data.shape, sample_labels.shape)
    return sample_data, sample_labels

def subset_data(data, labels, fraction=5):
    data_size = data.shape[0]
    out_size = int(data_size*fraction/100)
    idx = np.random.choice(data_size, out_size, replace=False)
    out_data = data[idx].copy()
    out_labels = labels[idx].copy()
    
    print(out_data.shape, out_labels.shape)
    return out_data, out_labels

In [4]:
# Load the raw CIFAR-10 data
cifar10_dir = 'lib/datasets/cifar-10-batches-py'
x_train, y_train, x_test, y_test = load_cifar10(cifar10_dir)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [5]:
testd = x_train[0:1000].copy()
testl = y_train[0:1000].copy()

In [6]:
def build_ganeval_model():
    K.clear_session()
    model_url = "https://tfhub.dev/deepmind/ganeval-cifar10-convnet/1"
    ganeval_module = thub.Module(model_url)
    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(32,32,3)))
    model.add(thub.KerasLayer(ganeval_module))
    model.add(tf.keras.layers.Activation('softmax'))
    
    model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.1, momentum=0.9, decay=1e-4),
                  loss=tf.keras.losses.CategoricalCrossentropy(),
                  metrics=['accuracy'])
    
    return model

In [9]:
ge_cifar_clf = build_ganeval_model()
ge_cifar_clf.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 10)                7796426   
_________________________________________________________________
activation (Activation)      (None, 10)                0         
Total params: 7,796,426
Trainable params: 0
Non-trainable params: 7,796,426
_________________________________________________________________


In [10]:
# Another ge_cifar model with more layers

K.clear_session()
#model_url = "https://tfhub.dev/deepmind/ganeval-cifar10-convnet/1"

ganeval_module_new = thub.Module(model_url)
# gan_eval_layer = thub.KerasLayer(model_url)


ge_cifar_new_clf = tf.keras.Sequential()
ge_cifar_new_clf.add(tf.keras.layers.InputLayer(input_shape=(32,32,3)))
ge_cifar_new_clf.add(thub.KerasLayer(ganeval_module_new))
ge_cifar_new_clf.add(tf.keras.layers.Activation('softmax'))
ge_cifar_new_clf.add(tf.keras.layers.Activation('relu'))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [11]:
ge_cifar_new_clf.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 10)                7796426   
_________________________________________________________________
activation (Activation)      (None, 10)                0         
_________________________________________________________________
activation_1 (Activation)    (None, 10)                0         
Total params: 7,796,426
Trainable params: 0
Non-trainable params: 7,796,426
_________________________________________________________________


In [12]:
ge_cifar_new_clf.compile(optimizer=tf.keras.optimizers.SGD(lr=0.1, momentum=0.9, decay=1e-4),
                  loss=tf.keras.losses.CategoricalCrossentropy(),
                  metrics=['accuracy'])

In [13]:
ge_cifar_new_clf.evaluate(testd, testl)

1000/1000 [==============================] - 3s 3ms/sample - loss: 0.0017 - acc: 1.0000


[0.00166802722774446, 1.0]

# Testing the retrieved hub model by evaluating on train data
Should give us 100% or something close to that

In [ ]:
ge_cifar_clf.evaluate(testd, testl)

# Testing a sample attack

In [ ]:
from art.attacks import FastGradientMethod

classifier = KerasClassifier(model=ge_cifar_clf, clip_values=(0, 1), use_logits=False)
attack_fgsm = FastGradientMethod(classifier=classifier, eps=0.4)

In [13]:
x_test_adv = attack_fgsm.generate(testd.copy())

In [14]:
ge_cifar_clf.evaluate(x_test_adv, testl)

1000/1000 [==============================] - 3s 3ms/sample - loss: 6.9334 - acc: 0.1020


[6.933382461547851, 0.102]

# Define the substitute classifier for our model

In [16]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

In [17]:
def build_substitute_model():
    model = tf.keras.Sequential( )
    model.add( Conv2D( 32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(32,32,3) ) )
    model.add( Conv2D( 64, (3, 3), padding='same', activation='relu' ) )
    model.add( MaxPooling2D( pool_size=(2, 2) ) )
    model.add( Flatten( ) )
    model.add( Dense( 128, activation='relu' ) )
    model.add( Dense(10, activation='softmax' ) )
    
    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                  optimizer='adam',
                  metrics=[ 'accuracy' ] )
    
    return model

In [18]:
sub_clf = build_substitute_model()

sub_clf.summary( )

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 16384)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               2097280   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 2,117,962
Trainable params: 2,117,962
Non-trainable params: 0
____________________________________________

In [19]:
x_train_adv, y_train_adv = sample_by_class(x_train, y_train, num_samples=100)
x_test_adv, y_test_adv = sample_by_class(x_test, y_test, num_samples=100)

(1000, 32, 32, 3) (1000, 10)
(1000, 32, 32, 3) (1000, 10)


In [20]:
partial_x_train, partial_y_train = subset_data(x_train, y_train, fraction=15)

(7500, 32, 32, 3) (7500, 10)


# Attacks

In [25]:
# from tensorflow.keras.models import Model
from tensorflow.keras.layers import InputLayer, Reshape
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation
from tensorflow.keras.losses import CategoricalCrossentropy

In [26]:
def build_ganeval_model(enable_logits=True):
    K.clear_session()
    model_url = "https://tfhub.dev/deepmind/ganeval-cifar10-convnet/1"
    ganeval_module = thub.Module(model_url)
    
    model = Sequential()
    model.add(InputLayer(input_shape=(32,32,3)))
    model.add(thub.KerasLayer(ganeval_module))
    if enable_logits:
        model.add(Activation('linear'))
    else: 
        model.add(Activation('softmax'))
        
    model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.1, momentum=0.9, decay=1e-4),
                  loss=CategoricalCrossentropy(from_logits=enable_logits),
                  metrics=['accuracy'])
    
    return model

In [27]:
ge_cifar_new_clf = build_ganeval_model()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [28]:
x_test_adv, y_test_adv = sample_by_class(x_test, y_test, num_samples=10)

(100, 32, 32, 3) (100, 10)


In [29]:
def applyAttack(attack_name, attack, model):
    x_test_adv_attack = attack.generate(x_test_adv.copy())
    loss ,acc = model.evaluate(x_test_adv_attack, y_test_adv)
    preds = model.predict(x_test_adv_attack)
    
    report = classification_report(np.argmax(y_test_adv, axis=1), 
                                   np.argmax(preds, axis=1), 
                                   output_dict=True)
    report = report['weighted avg']
    report.update({'Loss':loss})
    report.update({'Accuracy':acc})
    report.update({'Attack':attack_name})
    
    return report

In [24]:
results = []

# Pixel Attack

In [45]:
from art.attacks.evasion import PixelAttack
ge_cifar_new_clf = build_ganeval_model()
pixelA_clf = KerasClassifier(model=ge_cifar_new_clf, clip_values=(0, 1), use_logits=False)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [46]:
pixel_attack = PixelAttack(classifier = pixelA_clf, 
                         th=None, es=1, targeted=False, verbose=False)

In [47]:
results.append(applyAttack("Pixel Attack", pixel_attack, ge_cifar_new_clf))

/Library/anaconda3/lib/python3.7/site-packages/art/attacks/evasion/pixel_threshold.py:1136: RuntimeWarning: divide by zero encountered in double_scalars
  and self.callback(self._scale_parameters(self.population[0]), convergence=self.tol / convergence)


20/20 [==============================] - 0s 14ms/sample - loss: 200.0143 - acc: 0.0000e+00


/Library/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Threshold Attack - Not Working

In [ ]:
from art.attacks.evasion import ThresholdAttack
ge_cifar_new_clf = build_ganeval_model()
thresholdA_clf = KerasClassifier(model=ge_cifar_new_clf, clip_values=(0, 1), use_logits=False)

In [ ]:
threshold_attack = ThresholdAttack(classifier = thresholdA_clf, es=1, targeted=False, verbose=False)

In [ ]:
results.append(applyAttack("Threshold Attack", threshold_attack, ge_cifar_new_clf))

# HopSkipJump Attack - check

In [48]:
from art.attacks.evasion import HopSkipJump
ge_cifar_new_clf = build_ganeval_model()
hopSkipJump_clf = KerasClassifier(model=ge_cifar_new_clf, clip_values=(0, 1), use_logits=False)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [49]:
hopSkipJump_attack = HopSkipJump(classifier=hopSkipJump_clf, targeted=False, norm=2,
                                 max_iter=50, max_eval=10000, init_eval=10, init_size=10)
# classifier, targeted=False, norm=2, max_iter=50, max_eval=10000, init_eval=100, init_size=100

In [50]:
results.append(applyAttack("Hop Skip Jump Attack", hopSkipJump_attack, ge_cifar_new_clf))

20/20 [==============================] - 1s 30ms/sample - loss: 0.9367 - acc: 0.1000


/Library/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Projected Gradient Descent (PGD)

In [51]:
from art.attacks.evasion import ProjectedGradientDescent
ge_cifar_new_clf = build_ganeval_model()
projectedGradientDescent_clf = KerasClassifier(model=ge_cifar_new_clf, clip_values=(0, 1), use_logits=False)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [52]:
pgd_attack = ProjectedGradientDescent(classifier=projectedGradientDescent_clf, eps=0.4)

In [53]:
results.append(applyAttack("Projected Gradient Descent Attack", pgd_attack, ge_cifar_new_clf))

20/20 [==============================] - 0s 14ms/sample - loss: 50.7558 - acc: 0.0000e+00


/Library/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# NewtonFool

In [54]:
from art.attacks.evasion import NewtonFool
ge_cifar_new_clf = build_ganeval_model()
newtonFool_clf = KerasClassifier(model=ge_cifar_new_clf, clip_values=(0, 1), use_logits=False)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [55]:
newtonFool_attack = NewtonFool(classifier=newtonFool_clf, max_iter=100, eta=0.01, batch_size=1)

In [56]:
results.append(applyAttack("Newton Fool Attack", newtonFool_attack, ge_cifar_new_clf))

20/20 [==============================] - 0s 17ms/sample - loss: 2.4494 - acc: 0.0000e+00


/Library/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Elastic Net Attack

In [57]:
from art.attacks.evasion import ElasticNet
ge_cifar_new_clf = build_ganeval_model()
elasticNet_clf = KerasClassifier(model=ge_cifar_new_clf, clip_values=(0, 1), use_logits=False)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [58]:
elasticNet_attack = ElasticNet(classifier=elasticNet_clf, decision_rule='EN')

In [59]:
results.append(applyAttack("Elastic Net Attack", elasticNet_attack, ge_cifar_new_clf))

20/20 [==============================] - 0s 20ms/sample - loss: 3.8533 - acc: 0.0000e+00


/Library/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Spatial transformation attack

In [60]:
from art.attacks.evasion import SpatialTransformation
ge_cifar_new_clf = build_ganeval_model()
spatialT_clf = KerasClassifier(model=ge_cifar_new_clf, clip_values=(0, 1), use_logits=False)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [61]:
spatialT_attack = SpatialTransformation(classifier=spatialT_clf, max_translation=0.0, num_translations=1, max_rotation=0.0, num_rotations=1)

In [62]:
results.append(applyAttack("Spatial Transformation Attack", spatialT_attack, ge_cifar_new_clf))

20/20 [==============================] - 0s 14ms/sample - loss: 0.0122 - acc: 1.0000


# Zeroth-order optimization attack

In [63]:
from art.attacks.evasion import ZooAttack
ge_cifar_new_clf = build_ganeval_model()
zoo_clf = KerasClassifier(model=ge_cifar_new_clf, clip_values=(0, 1), use_logits=False)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [64]:
zoo_attack = ZooAttack(classifier=zoo_clf,confidence=0.0, targeted=False, learning_rate=0.01, max_iter=10, 
                              binary_search_steps=1, initial_const=0.001, abort_early=True, use_resize=True, 
                              use_importance=True, nb_parallel=128, batch_size=1, variable_h=0.0001)

In [65]:
results.append(applyAttack("Zeroth-Order Optimization Attack", zoo_attack, ge_cifar_new_clf))

20/20 [==============================] - 0s 15ms/sample - loss: 0.0122 - acc: 1.0000


# Carlini and Wagner L_inf Attack

In [66]:
from art.attacks.evasion import CarliniLInfMethod
ge_cifar_new_clf = build_ganeval_model()
carliniLInf_clf = KerasClassifier(model=ge_cifar_new_clf, clip_values=(0, 1), use_logits=False)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [67]:
carliniLInf_attack = CarliniLInfMethod(classifier=carliniLInf_clf, confidence=0.0, targeted=False, 
                                       learning_rate=0.01, max_iter=10, max_halving=5, max_doubling=5, 
                                       eps=0.3, batch_size=128)

In [68]:
results.append(applyAttack("Carlini and Wagner L_inf Attack", carliniLInf_attack, ge_cifar_new_clf))

20/20 [==============================] - 1s 25ms/sample - loss: 3.2943 - acc: 0.0500


/Library/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Jacobian Saliency Map Attack

In [69]:
from art.attacks.evasion import SaliencyMapMethod
ge_cifar_new_clf = build_ganeval_model()
saliencyMap_clf = KerasClassifier(model=ge_cifar_new_clf, clip_values=(0, 1), use_logits=False)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [70]:
saliency_attack = SaliencyMapMethod(classifier=saliencyMap_clf, theta=0.1, gamma=1.0, batch_size=50)

In [71]:
results.append(applyAttack("Jacobian Saliency Map Attack", saliency_attack, ge_cifar_new_clf))

20/20 [==============================] - 0s 13ms/sample - loss: 1.1631 - acc: 0.0000e+00


/Library/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Universal perturbation Moosavi

In [72]:
from art.attacks.evasion import UniversalPerturbation
ge_cifar_new_clf = build_ganeval_model()
universalPerturbation_clf = KerasClassifier(model=ge_cifar_new_clf, clip_values=(0, 1), use_logits=False)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [73]:
universalPerturbation_attack = UniversalPerturbation(classifier=universalPerturbation_clf, attacker='deepfool', attacker_params=None, delta=0.2, max_iter=20, eps=10.0, norm=2)

In [74]:
results.append(applyAttack("Universal perturbation Moosavi Attack", universalPerturbation_attack, ge_cifar_new_clf))

art.attacks.evasion.deepfool.DeepFool
20/20 [==============================] - 0s 12ms/sample - loss: 6.6463 - acc: 0.1000


/Library/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# DeepFool attack

In [75]:
from art.attacks.evasion import DeepFool
ge_cifar_new_clf = build_ganeval_model()
deepFool_clf = KerasClassifier(model=ge_cifar_new_clf, clip_values=(0, 1), use_logits=False)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [76]:
deepFool_attack = DeepFool(classifier=deepFool_clf, max_iter=100, epsilon=1e-06, nb_grads=10, batch_size=1)

In [77]:
results.append(applyAttack("Deep Fool Attack", deepFool_attack, ge_cifar_new_clf))

20/20 [==============================] - 0s 13ms/sample - loss: 5.8041 - acc: 0.1000


/Library/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Virtual Adversarial Method

In [ ]:
from art.attacks.evasion import VirtualAdversarialMethod
ge_cifar_new_clf = build_ganeval_model()
virtualAdversarial_clf = KerasClassifier(model=ge_cifar_new_clf, clip_values=(0, 1), use_logits=False)

In [ ]:
#virtualAdversarial_attack = VirtualAdversarialMethod(classifier=virtualAdversarial_clf, max_iter=100, nb_grads=10, batch_size=100)

virtualAdversarial_attack = VirtualAdversarialMethod(classifier=virtualAdversarial_clf, max_iter=100, finite_diff=1e-06, eps=0.1, batch_size=1)

In [ ]:
results.append(applyAttack("Virtual Adversarial Method Attack", virtualAdversarial_attack, ge_cifar_new_clf))

# Fast Gradient Method Attack

In [79]:
from art.attacks.evasion import FastGradientMethod
ge_cifar_new_clf = build_ganeval_model()
fgm_clf = KerasClassifier(model=ge_cifar_new_clf, clip_values=(0, 1), use_logits=False)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [80]:
fgm_attack = FastGradientMethod(classifier=fgm_clf, eps=0.3, eps_step=0.1, targeted=False, 
                                num_random_init=0, batch_size=1, minimal=False)
# classifier, norm=inf, eps=0.3, eps_step=0.1, targeted=False, num_random_init=0, batch_size=1, minimal=False

In [81]:
results.append(applyAttack("Fast Gradient Method", fgm_attack, ge_cifar_new_clf))

20/20 [==============================] - 0s 9ms/sample - loss: 5.9313 - acc: 0.0500


/Library/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Carlini and Wagner L_2 Attack

In [85]:
from art.attacks.evasion import CarliniL2Method
ge_cifar_new_clf = build_ganeval_model()

carliniL2_clf = KerasClassifier(model=ge_cifar_new_clf, clip_values=(0, 1), use_logits=False)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [86]:
carliniL2_attack = CarliniL2Method(classifier=carliniL2_clf, confidence=0.0, targeted=False, learning_rate=0.01, binary_search_steps=10, max_iter=10, initial_const=0.01, max_halving=5, max_doubling=5, batch_size=1)

In [87]:
results.append(applyAttack("Carlini and Wagner L_2 Attack", carliniL2_attack, ge_cifar_new_clf))

20/20 [==============================] - 0s 17ms/sample - loss: 1.3306 - acc: 0.3000


/Library/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# NOT WORKING ATTACKS

# Adversarial Patch

In [ ]:
from art.attacks.evasion import AdversarialPatch
ge_cifar_new_clf = build_ganeval_model()

advPatch_clf = KerasClassifier(model=ge_cifar_new_clf, clip_values=(0, 1), use_logits=False)

advPatch_attack = AdversarialPatch(classifier=advPatch_clf)

In [ ]:
results.append(applyAttack("Adversarial Patch Attack", advPatch_attack, ge_cifar_new_clf))

# Decision-Based / Boundary Attack

In [ ]:
from art.attacks.evasion import BoundaryAttack
ge_cifar_new_clf = build_ganeval_model()

boundary_clf = KerasClassifier(model=ge_cifar_new_clf, clip_values=(0, 1), use_logits=False)

boundary_attack = BoundaryAttack(classifier=boundary_clf, targeted=False)

In [ ]:
results.append(applyAttack("Decision-Based / Boundary Attack", boundary_attack, ge_cifar_new_clf))

# Query-Efficient Black-Box Attack -> Different Implementation

In [ ]:
from art.wrappers import QueryEfficientBBGradientEstimation
ge_cifar_new_clf = build_ganeval_model()
queryEffBlackBox_clf = KerasClassifier(model=ge_cifar_new_clf, clip_values=(0, 1), use_logits=False)

In [ ]:
qeBlackBox_attack = QueryEfficientBBGradientEstimation(classifier=queryEffBlackBox_clf, num_basis=0, sigma=0, round_samples=0)
# classifier, num_basis, sigma, round_samples=0

In [ ]:
qeBlackBox_attack.fit(x_test_adv, y_test_adv)

In [ ]:
qeBlackBox_lossGradient = qeBlackBox_attack.loss_gradient(x_test_adv, y_test_adv)

In [ ]:
qeBB_prediction = qeBlackBox_attack.predict(x_test_adv, y_test_adv)

# Final Results of Attacks

In [88]:
results

[{'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 20.0,
  'Loss': 200.01431274414062,
  'Accuracy': 0.0,
  'Attack': 'Pixel Attack'},
 {'precision': 0.02222222222222222,
  'recall': 0.1,
  'f1-score': 0.03636363636363636,
  'support': 20,
  'Loss': 0.9367471933364868,
  'Accuracy': 0.1,
  'Attack': 'Hop Skip Jump Attack'},
 {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 20.0,
  'Loss': 50.75578308105469,
  'Accuracy': 0.0,
  'Attack': 'Projected Gradient Descent Attack'},
 {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 20.0,
  'Loss': 2.4494290351867676,
  'Accuracy': 0.0,
  'Attack': 'Newton Fool Attack'},
 {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 20.0,
  'Loss': 3.853330135345459,
  'Accuracy': 0.0,
  'Attack': 'Elastic Net Attack'},
 {'precision': 1.0,
  'recall': 1.0,
  'f1-score': 1.0,
  'support': 20,
  'Loss': 0.012248705141246319,
  'Accuracy': 1.0,
  'Attack': 'Spatial Transformation Attack

In [89]:
import pandas as pd
df = pd.DataFrame(results)
df

,precision,recall,f1-score,support,Loss,Accuracy,Attack
0,0.000000,0.00,0.000000,20.0,200.014313,0.00,Pixel Attack
1,0.022222,0.10,0.036364,20.0,0.936747,0.10,Hop Skip Jump Attack
2,0.000000,0.00,0.000000,20.0,50.755783,0.00,Projected Gradient Descent Attack
3,0.000000,0.00,0.000000,20.0,2.449429,0.00,Newton Fool Attack
4,0.000000,0.00,0.000000,20.0,3.853330,0.00,Elastic Net Attack
5,1.000000,1.00,1.000000,20.0,0.012249,1.00,Spatial Transformation Attack
6,1.000000,1.00,1.000000,20.0,0.012249,1.00,Zeroth-Order Optimization Attack
7,0.020000,0.05,0.028571,20.0,3.294347,0.05,Carlini and Wagner L_inf Attack
8,0.000000,0.00,0.000000,20.0,1.163113,0.00,Jacobian Saliency Map Attack
9,0.011765,0.10,0.021053,20.0,6.646263,0.10,Universal perturbation Moosavi Attack


In [90]:
df.to_csv('AttacksResults20.csv')

# Query-Efficient Black-Box Attack

In [ ]:
from art.wrappers import QueryEfficientBBGradientEstimation
queryEffBlackBox_clf = KerasClassifier(model=ge_cifar_clf, clip_values=(0, 1), use_logits=False)

In [ ]:
qeBlackBox_attack = QueryEfficientBBGradientEstimation(classifier=queryEffBlackBox_clf, num_basis=0, sigma=0, round_samples=0)
# classifier, num_basis, sigma, round_samples=0

In [ ]:
qeBlackBox_attack.fit(testd[:10], testl[:10])

In [ ]:
qeBlackBox_lossGradient = qeBlackBox_attack.loss_gradient(testd[:10], testl[:10])

In [ ]:
qeBB_prediction = qeBlackBox_attack.predict(x_test[:10])

# Defences

# Gaussian Data Augmentation Defence

In [109]:
from art.defences.preprocessor import GaussianAugmentation
## do something

In [110]:
gaussian_defence = GaussianAugmentation(sigma=1.0, augmentation=True, ratio=1.0, clip_values=None, apply_fit=True, apply_predict=False)

In [113]:
gaus_aug_data = gaussian_defence.__call__(x=x_test_adv[:50], y=y_train_adv[:50])
gaus_aug_data

(array([[[[ 6.07843137e-01,  6.11764706e-01,  5.84313725e-01],
          [ 6.54901961e-01,  6.90196078e-01,  7.33333333e-01],
          [ 6.90196078e-01,  7.01960784e-01,  7.56862745e-01],
          ...,
          [ 7.88235294e-01,  7.68627451e-01,  8.19607843e-01],
          [ 7.92156863e-01,  7.92156863e-01,  8.31372549e-01],
          [ 7.52941176e-01,  7.17647059e-01,  6.70588235e-01]],
 
         [[ 6.00000000e-01,  6.07843137e-01,  6.15686275e-01],
          [ 6.39215686e-01,  7.01960784e-01,  8.00000000e-01],
          [ 6.70588235e-01,  7.21568627e-01,  8.43137255e-01],
          ...,
          [ 7.92156863e-01,  8.15686275e-01,  8.70588235e-01],
          [ 7.96078431e-01,  8.43137255e-01,  8.90196078e-01],
          [ 7.41176471e-01,  7.45098039e-01,  7.17647059e-01]],
 
         [[ 6.07843137e-01,  6.03921569e-01,  6.00000000e-01],
          [ 6.27450980e-01,  6.98039216e-01,  7.88235294e-01],
          [ 6.58823529e-01,  7.25490196e-01,  8.35294118e-01],
          ...,
    

In [114]:
est_grad = gaussian_defence.estimate_gradient(x_test_adv[:50], grad = 0.001)
est_grad

0.001

In [ ]:
gaussian_defence

# Feature Squeezing

In [115]:
from art.defences.preprocessor import FeatureSqueezing

In [119]:
featureSqueezing_defence = FeatureSqueezing(clip_values=(0,255), bit_depth=8, apply_fit=True, apply_predict=False)
# clip_values = (min,max), bit_depth=8, apply_fit=False, apply_predict=True
featureSqueezing_defence

In [120]:
est_grad_featSqz = featureSqueezing_defence.estimate_gradient(x_test_adv[:50], grad = 0.001)
est_grad_featSqz

0.001

# High Confidence Defence

In [122]:
from art.defences.postprocessor import HighConfidence

In [124]:
highConf_defence = HighConfidence(cutoff=0.25, apply_fit=False, apply_predict=True)
highConf_defence

In [125]:
highConf_defence.__call__(x_test_adv[:50])

array([[[[0.60784314, 0.61176471, 0.58431373],
         [0.65490196, 0.69019608, 0.73333333],
         [0.69019608, 0.70196078, 0.75686275],
         ...,
         [0.78823529, 0.76862745, 0.81960784],
         [0.79215686, 0.79215686, 0.83137255],
         [0.75294118, 0.71764706, 0.67058824]],

        [[0.6       , 0.60784314, 0.61568627],
         [0.63921569, 0.70196078, 0.8       ],
         [0.67058824, 0.72156863, 0.84313725],
         ...,
         [0.79215686, 0.81568627, 0.87058824],
         [0.79607843, 0.84313725, 0.89019608],
         [0.74117647, 0.74509804, 0.71764706]],

        [[0.60784314, 0.60392157, 0.6       ],
         [0.62745098, 0.69803922, 0.78823529],
         [0.65882353, 0.7254902 , 0.83529412],
         ...,
         [0.80784314, 0.81568627, 0.84313725],
         [0.8       , 0.84313725, 0.87843137],
         [0.74117647, 0.74901961, 0.72156863]],

        ...,

        [[0.34901961, 0.30980392, 0.        ],
         [0.        , 0.        , 0.        ]

# Rounding/Rounded Defence

In [126]:
from art.defences.postprocessor import Rounded

In [127]:
rounded_defence = Rounded(decimals=3, apply_fit=False, apply_predict=True)

In [128]:
rounded_defence.__call__(x_test_adv[:50])

array([[[[0.608, 0.612, 0.584],
         [0.655, 0.69 , 0.733],
         [0.69 , 0.702, 0.757],
         ...,
         [0.788, 0.769, 0.82 ],
         [0.792, 0.792, 0.831],
         [0.753, 0.718, 0.671]],

        [[0.6  , 0.608, 0.616],
         [0.639, 0.702, 0.8  ],
         [0.671, 0.722, 0.843],
         ...,
         [0.792, 0.816, 0.871],
         [0.796, 0.843, 0.89 ],
         [0.741, 0.745, 0.718]],

        [[0.608, 0.604, 0.6  ],
         [0.627, 0.698, 0.788],
         [0.659, 0.725, 0.835],
         ...,
         [0.808, 0.816, 0.843],
         [0.8  , 0.843, 0.878],
         [0.741, 0.749, 0.722]],

        ...,

        [[0.349, 0.31 , 0.243],
         [0.224, 0.208, 0.204],
         [0.239, 0.239, 0.231],
         ...,
         [0.349, 0.467, 0.396],
         [0.231, 0.31 , 0.243],
         [0.263, 0.29 , 0.196]],

        [[0.349, 0.322, 0.243],
         [0.235, 0.227, 0.208],
         [0.247, 0.259, 0.227],
         ...,
         [0.224, 0.259, 0.227],
         [0.

# Virtual Adversarial Training Defence

In [103]:
from art.defences.trainer import AdversarialTrainer
advTrainer_clf = KerasClassifier(model=ge_cifar_new_clf, clip_values=(0, 1), use_logits=False)

In [104]:
adversarialTrainer_defence = AdversarialTrainer(classifier=advTrainer_clf, attacks=fgm_attack, ratio=0.5)

In [105]:
adversarialTrainer_defence.fit(x_train_adv[:50].copy(), y_train_adv[:50].copy())

1/1 [==============================] - 0s 139ms/step - loss: 3.1076 - acc: 0.4800


In [106]:
at_predictions = adversarialTrainer_defence.predict(x_test_adv[:50].copy())